In [1]:
import json
import importlib
import os
current_dir=os.getcwd()
os.chdir("../../")
# print(os.getcwd())
import dist_utilities as dist_ult
# import noise_est_utilities as ne_ult
importlib.reload(dist_ult)
# importlib.reload(ne_ult)
os.chdir(current_dir)
import pickle
from qiskit.result import Result
from qiskit.visualization import plot_histogram, plot_distribution
import matplotlib.pyplot as plt
import numpy as np
from copy import deepcopy
from qiskit.result import QuasiDistribution

In [ ]:
with open("ideal_counts_sim.json", "r") as f:
    ideal_counts=json.load(f)
with open("noisy_counts_sim.json", "r") as f:
    noisy_counts=json.load(f)
with open("noisy_noise_est_dist_sim.json", "r") as f:
    noisy_noise_est_dist=json.load(f)
with open("ideal_noise_est_counts_sim.json", "r") as f:
    ideal_noise_est_counts=json.load(f)
print("ideal: ", ideal_noise_est_counts)
print("noise est dist: ", noisy_noise_est_dist)
print("noisy counts: ", noisy_counts)
print("ideal counts: ", ideal_counts)
col_label=list(dict(ideal_noise_est_counts).keys())[0]
# corrected_counts=ne_ult.get_corrected_counts_from_counts(noisy_counts, noisy_noise_est_dist, list(ideal_noise_est_counts.keys())[0])

# print(corrected_counts)

In [33]:
# plot_histogram([ideal_counts, noisy_counts, corrected_counts], number_to_keep=5, legend=["ideal counts", "noisy counts", "corrected counts"], figsize=(20,20), filename="plot_sim.jpg")


In [ ]:
num_qubits=len(list(noisy_counts.items())[0][0])

if 2**15<2**num_qubits:
    num_keep=2**10#num_qubits#2**10
else:
    num_keep=2**num_qubits
noisy_counts_orig=deepcopy(noisy_counts)
print("num keep ", num_keep)
noisy_noise_est_dist=dist_ult.convert_to_first_col(noisy_noise_est_dist, col_label)
noisy_noise_est_dist=dist_ult.filter_counts(noisy_noise_est_dist, num_keep)
noisy_counts=dist_ult.filter_counts(noisy_counts, num_keep)
all_keys, noisy_noise_est_dist, noisy_counts=dist_ult.fill_counts_range(noisy_noise_est_dist, noisy_counts, num_keep, num_qubits)
print(noisy_counts)
print(noisy_noise_est_dist)

noisy_noise_est_vec=sorted(list(noisy_noise_est_dist.items()), key=lambda x: int(x[0], 2))
noisy_noise_est_vec=[elem[1] for elem in noisy_noise_est_vec]
counts_vec=sorted(list(noisy_counts.items()), key=lambda x: int(x[0], 2))
counts_vec=[elem[1] for elem in counts_vec]
# print(len(noisy_noise_est_vec))
# print(len(counts_vec))
quasi_corrected_counts=dist_ult.get_quasi_corrected(noisy_noise_est_vec, counts_vec)
quasi_corrected_counts=dist_ult.label_quasis(quasi_corrected_counts, all_keys)
# print(quasi_corrected_counts)
quasi_method=QuasiDistribution(quasi_corrected_counts)
quasi_corrected_counts=quasi_method.nearest_probability_distribution().binary_probabilities(num_qubits)
# print(quasi_corrected_counts)
# quasi_corrected_counts={bin(idx)[2::].zfill(num_qubits):val for idx, val in enumerate(quasi_corrected_counts)}



In [35]:
fig=plot_histogram([ideal_counts, noisy_counts_orig, quasi_corrected_counts], number_to_keep=15, #bar_labels=False,
               legend=["ideal", "raw", "corrected"], title="Dicke n=20, k=1 Simulation", bar_labels=False)
fig.savefig("plot_dicke20_1_counts_sim.jpg", bbox_inches="tight")

In [36]:
fig=plot_distribution([ideal_counts, noisy_counts_orig, quasi_corrected_counts], number_to_keep=15, #bar_labels=False,
                  legend=["ideal", "raw", "corrected"], title="Dicke n=20, k=1 Simulation", bar_labels=False) #figsize=(15,15))
ax=fig.axes[0]
ax.set_ylabel("Probability")
fig.tight_layout()
fig.savefig("plot_dicke20_1_dist_sim.jpg", bbox_inches="tight")